In [6]:
import rasterio as rio
from rasterio.mask import mask
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs
from os import path, remove
import pyproj
import numpy as np
from xml.dom import minidom
from shapely.geometry import Point, mapping

from glob import glob
import re
from subprocess import call 

# Image Data Extraction Pipeline
The purpose here is to get all of the images in a directory, extract an area of interest from them, get pixel values, and add a row to a database for each pixel in the area of interest within each image. 

In [7]:
IMAGEDIR = "../images/2016/snow"
locfile = "these_locations.csv"
outfilename = "PROVISIONAL_2016_snow_datatable.csv"
TOA_ADJUST = False
file_search_format = "*_*.zip"
filename_regex = "(\d*)_(.*).zip"

In [8]:
files = list(glob(path.join(IMAGEDIR, file_search_format)))
locations = pd.read_csv(path.join(IMAGEDIR, locfile)).set_index("id")

In [12]:
def process_file(fullpath):
    
    filename = path.basename(fullpath)
    match = re.search(filename_regex, filename)
    loc_id = match[1]
    img_id = match[2]
    print("Unzipping..."+fullpath)
    call(['unzip', fullpath])

    loc = locations.loc[int(loc_id)]
    # Read File
    print(glob("*AnalyticMS_clip.tif")[0])
    raster = rio.open(glob("*AnalyticMS_clip.tif")[0])

    # Get TOA Coefficeints from metadata
    meta = minidom.parse(glob("*AnalyticMS_metadata_clip.xml")[0])
    nodes = meta.getElementsByTagName("ps:bandSpecificMetadata")

    coeffs = {}
    for node in nodes:
        bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
        if bn in ['1', '2', '3', '4']:
            i = int(bn)
            value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
            coeffs[i] = float(value)
            
    rast_proj = pyproj.Proj(init=raster.crs['init'])
    buf_proj = pyproj.Proj(init='epsg:4326')
    
    lon, lat = pyproj.transform(buf_proj, rast_proj, loc.longitude, loc.latitude)
    buf = mapping(Point(lon, lat).buffer(5))
    
    print("{lo} {la} --> {ulo} {ula}".format(lo=loc.longitude, la=loc.latitude,
                                             ulo=lon, ula=lat))
    print(raster.bounds)
    
    try: 
        out_image, out_transform = mask(raster, [buf], crop=True)
    except Exception as e:
        print(e) 
        for f in glob(img_id+"*"):
            print("Deleting " + f)
            remove(f)
        raise Exception

    no_data = raster.nodata
    data = out_image.data[0]
    
    reflectance = np.extract(data != no_data, data)
    row, col = np.where(data != no_data) 
    T1 = out_transform * rio.Affine.translation(0.5, 0.5) # reference the pixel centre
    rc2xy = lambda r, c: (c, r) * T1 
    
    d = gpd.GeoDataFrame({'col':col,'row':row, 'loc':loc_id, 'img': img_id})
    # coordinate transformation
    try:
        d['x_geotiff'] = d.apply(lambda row: rc2xy(row.row,row.col)[0], axis=1)
        d['y_geotiff'] = d.apply(lambda row: rc2xy(row.row,row.col)[1], axis=1)
        d['geometry'] =d.apply(lambda row: Point(row['x_geotiff'], row['y_geotiff']), axis=1)
        for band in range(4):
            # read + TOA normalize
            data = out_image.data[band]    
            if(TOA_ADJUST):
                reflectance = np.extract(data != no_data, data) * coeffs[band + 1] 
            else:
                reflectance = np.extract(data != no_data, data)
            d['band'+str(band+1)] = pd.Series(reflectance, index=d.index)
    except Exception as e:
        for f in glob(img_id+"*"):
            print("Deleting " + f)
            remove(f)
        print(e)
        print("Error reading, aborted.")
        raise Exception("Error reading")

        
    d.crs = raster.crs
    d = d.to_crs({'init':'epsg:4326'})
    
        
    
    for f in glob(img_id+"*"):
        print("Deleting " + f)
        remove(f)
    
    print('done')
    return(d.drop(['row', 'col'], axis=1))
    


In [13]:
dfs = []
for file in files:
    print(file)
    try:
        ans = process_file(file)
    except Exception as e:
        print(e)
        print("Error, skipping")
        continue
    dfs.append(ans)
    
dfs = pd.concat(dfs)

../images/2016/snow/1882_20170119_181700_0e20.zip
Unzipping...../images/2016/snow/1882_20170119_181700_0e20.zip
20170119_181700_0e20_3B_AnalyticMS_clip.tif
-121.8872 46.99467 --> 584609.311164629 5205172.702328316
BoundingBox(left=584220.0, bottom=5204613.0, right=584997.0, top=5205735.0)
Deleting 20170119_181700_0e20_3B_AnalyticMS_DN_udm_clip.tif
Deleting 20170119_181700_0e20_3B_AnalyticMS_clip.tif
Deleting 20170119_181700_0e20_3B_AnalyticMS_metadata_clip.xml
done
../images/2016/snow/1882_20170105_174048_0c45.zip
Unzipping...../images/2016/snow/1882_20170105_174048_0c45.zip
20170105_174048_0c45_3B_AnalyticMS_clip.tif
-121.8872 46.99467 --> 584609.311164629 5205172.702328316
BoundingBox(left=584220.0, bottom=5204613.0, right=584997.0, top=5205735.0)
Deleting 20170105_174048_0c45_3B_AnalyticMS_metadata_clip.xml
Deleting 20170105_174048_0c45_3B_AnalyticMS_clip.tif
Deleting 20170105_174048_0c45_3B_AnalyticMS_DN_udm_clip.tif
done
../images/2016/snow/1882_20161206_181601_0e19.zip
Unzipping.

In [13]:
dfs.to_csv(path.join(IMAGEDIR, outfilename))

In [14]:
len(dfs)

76

In [15]:
dfs

,img,loc,x_geotiff,y_geotiff,geometry,band1,band2,band3,band4
0,20170119_181700_0e20,1882,584605.5,5205175.5,POINT (-121.8872495930271 46.99469565792243),3122,2770,2004,1342
1,20170119_181700_0e20,1882,584608.5,5205175.5,POINT (-121.8872101439792 46.99469527452418),3107,2755,1971,1332
2,20170119_181700_0e20,1882,584611.5,5205175.5,POINT (-121.8871706949321 46.99469489111234),3086,2743,1974,1348
3,20170119_181700_0e20,1882,584605.5,5205172.5,POINT (-121.8872501533729 46.99466866670392),3116,2738,1975,1339
4,20170119_181700_0e20,1882,584608.5,5205172.5,POINT (-121.8872107043449 46.99466828330603),3076,2731,1946,1313
5,20170119_181700_0e20,1882,584611.5,5205172.5,POINT (-121.8871712553176 46.99466789989455),3069,2747,1964,1300
6,20170119_181700_0e20,1882,584605.5,5205169.5,POINT (-121.887250713718 46.9946416754853),3165,2745,1999,1368
7,20170119_181700_0e20,1882,584608.5,5205169.5,POINT (-121.8872112647098 46.99464129208776),3124,2743,1976,1345
8,20170119_181700_0e20,1882,584611.5,5205169.5,POINT (-121.8871718157024 46.99464090867664),3128,2772,1984,1317
0,20170105_174048_0c45,1882,584605.5,5205175.5,POINT (-121.8872495930271 46.99469565792243),1846,1222,750,345
